In [1]:
import cv2

In [2]:
import numpy as np

In [3]:
from picamera.array import PiRGBArray
from picamera import PiCamera

In [4]:
camera = PiCamera()

In [5]:
def create_fullscreen_window():
    cv2.namedWindow("fw", cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty("fw",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)

In [27]:
def show_image(im, timeout=1000):
    im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
    cv2.imshow("fw", im)
    cv2.waitKey(timeout)

In [7]:
def cleanup_window():
    cv2.destroyWindow("fw")

In [8]:
def get_image():
    rawCapture = PiRGBArray(camera)
    camera.capture(rawCapture, format="rgb")
    image = rawCapture.array
    return image

In [12]:
def scale_image(image):
    small_image = cv2.resize(image[0:480, 160:640], dsize=(299,299))
    small_image_f32 = np.float32(small_image)
    small_image_f32_r3 = np.expand_dims(small_image_f32, 0)
    scaled_image = small_image_f32_r3/255
    return scaled_image

In [13]:
from tensorflow import keras

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [14]:
import tensorflow as tf

In [15]:
iv3 = keras.applications.InceptionV3(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000
)

In [16]:
def predict_image(im):
    prediction = iv3.predict(im)
    return keras.applications.inception_v3.decode_predictions(prediction)

In [17]:
def predict(im):
    prediction = predict_image(im)
    name = prediction[0][1][1]
    return name

In [18]:
import subprocess
import shlex
import tempfile
import os
import time

In [19]:
def say_something_better(text):
    number, fname = tempfile.mkstemp(suffix='.wav')
    subprocess.call(["pico2wave", "--wave={}".format(fname), "-len-GB", text])
    subprocess.Popen(["aplay", "{}".format(fname)])
    os.remove(fname)

In [20]:
import json

with open('./joern_api_responses.json', 'r') as f:
    thought_responses = json.load(f)

In [21]:
def category_to_thought(category):
    retval = None
    try:
        retval = thought_responses[category]
    except:
        retval = [category, "Beer"]
    return retval

In [69]:
def puttext(im, text, scale=1, thickness=1, top=True):
    text = text.strip('. ')
    h, w, _ = im.shape
    x, y = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, scale, thickness)[0]
    xpos = (w // 2) - (x // 2)
    if top:
        ypos = 30
    else:
        ypos = h - 10
    im = cv2.putText(im, text, (xpos, ypos), cv2.FONT_HERSHEY_SIMPLEX, scale, (0, 0, 0), thickness+2)
    im = cv2.putText(im, text, (xpos, ypos), cv2.FONT_HERSHEY_SIMPLEX, scale, (255, 255, 255), thickness)
    return im

In [70]:
def main_loop():
    create_fullscreen_window()
    for i in range(5):
        im = get_image()
        stimulus = predict(scale_image(im))
        stimulus, response = category_to_thought(stimulus)
        string_to_speak = ". When I see {}, I think of {}.".format(stimulus, response)
        string_to_speak = string_to_speak.replace("_", " ")
        print(string_to_speak)
        string_top, string_bottom = string_to_speak.split(', ')
        im = puttext(im, string_top)
        im = puttext(im, string_bottom, top=False)
        show_image(im)
        say_something_better(string_to_speak)
        # time.sleep(10)
    cleanup_window()

In [71]:
main_loop()

. When I see Spatula, I think of Film.
. When I see Nipple, I think of Milk.
. When I see Tub, I think of Technology.
. When I see Band Aid, I think of 10.
. When I see Bathtub, I think of House.
